Requirements
+ python (preferably 3.9>)
+ selenium
+ bs4
+ A linkedin account!
+ chromedriver.exe on your local machine

# Constants:

In [47]:
MY_DRIVER_PATH = "C:/chromedriver.exe"
DB_NAME = "connections.db"
LOG_NAME = "Linkedin.log"
USER_NAME = "Your_User_name"
PASSWORD = "Your_Password"
SING_IN_PAGE = "https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin"
CONNECTIONS_PAGE = "https://www.linkedin.com/mynetwork/invite-connect/connections/"
MY_PAGE = "https://www.linkedin.com/in/mohammadreza-ardestani/"

# Imports

In [12]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.edge.service import Service

import time 

from bs4 import BeautifulSoup

import sqlite3

import logging

# Subroutines

In [45]:
def dump_data(data):
    logging.info(" *** Start dumping data to sqlite3  *** ")
    con = sqlite3.connect('connections.db')
    cur = con.cursor()
    cur.execute("""CREATE TABLE people(
                name text,
                occupation text,
                location text,
                link text
                )""")
    for lst in data: # lst is a list of information for one person
        try:
            cur.execute("INSERT INTO people VALUES (?,?,?,?)",(lst[0],lst[1],lst[2],lst[3]))
            con.commit()
        except:
            logging.error(f'+++info of person:{lst[0]} had problem while dumpping it+++')
    
    con.close()        
    mess = "dummped all contacts info Successfully"
    print(mess)    
    logging.info(mess)

In [40]:
def crawler(driver, links):
    info = []
    i = -1
    for lk in links:
        i+=1
        try:
            driver.get(lk)
            time.sleep(5)
            src = driver.page_source
            tmp = extract_info(src,lk)
            info.append(tmp)
            logging.info(f'id:{i} Successfully logged user:{tmp[0]}\n\n')
        except:
            logging.error(f'id:{i} +++ Error +++ raised for first try for user:{tmp[3]}\n\n')
            try:
                driver = build_driver()
                sing_in(driver)
                driver.get(lk)
                time.sleep(5)
                src = driver.page_source
                tmp = extract_info(src,lk)
                info.append(tmp)
                logging.info(f'id:{i} Successfully logged user:{tmp[0]}\n\n')
            except:
                logging.error(f'id:{i} +++++ second time Error ++++++ \n\n')
                return -1
            
        
    logging.info("SUCCESSFULLY LOGGED ALL USERS")
    return info

In [37]:
def extract_info(src,lk):
    tmp = ["" for _ in range(0,4)]
    tmp[3] = lk
    soup = BeautifulSoup(src, "html.parser")
    intro = soup.find('div', {'class': 'pv-text-details__left-panel'})
    name_loc = intro.find("h1")
    name = name_loc.get_text().strip()
    works_at_loc = intro.find("div", {'class': 'text-body-medium'})
    works_at = works_at_loc.get_text().strip()
    intro2 = soup.find('div', {'class': 'pb2 pv-text-details__left-panel'})
    location_loc = intro2.find_all("span", {'class': 'text-body-small'})
    location = location_loc[0].get_text().strip()
    tmp[0],tmp[1],tmp[2] = name,works_at,location
    return tmp
    

In [38]:
def grab_links(driver):
    sing_in(d)
    driver.get(CONNECTIONS_PAGE)
    time.sleep(30)
    lnks=driver.find_elements_by_tag_name("a")
    connections_links = []
    for lnk in lnks[8:210]: # out of this range is useless 
        connections_links.append(lnk.get_attribute('href'))
    # remove dublicates
    del connections_links[1::2]
    connections_links.append(MY_PAGE)
    return connections_links


In [39]:
def sing_in(driver):
    driver.get(SING_IN_PAGE)
    time.sleep(5)
    # sing in
    username = driver.find_element_by_xpath("//input[@name='session_key']")
    password = driver.find_element_by_xpath("//input[@name='session_password']")

    username.send_keys(USER_NAME)
    password.send_keys(PASSWORD)
    time.sleep(5)
    submit = driver.find_element_by_xpath("//button[@type='submit']").click()
    time.sleep(5)

In [18]:
def build_driver():
    ser = Service(MY_DRIVER_PATH)
    op = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=ser, options=op)
    return driver

# Driver code (aka main function)

In [32]:
d = build_driver()

In [33]:
connections_links = grab_links(d)

C:\Users\Mohamadreza.a\AppData\Local\Temp\ipykernel_16372\2719322194.py:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  username = driver.find_element_by_xpath("//input[@name='session_key']")
C:\Users\Mohamadreza.a\AppData\Local\Temp\ipykernel_16372\2719322194.py:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  password = driver.find_element_by_xpath("//input[@name='session_password']")
C:\Users\Mohamadreza.a\AppData\Local\Temp\ipykernel_16372\2719322194.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  submit = driver.find_element_by_xpath("//button[@type='submit']").click()
C:\Users\Mohamadreza.a\AppData\Local\Temp\ipykernel_16372\1550826900.py:5: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  lnks=driver.find_elements_by_tag_name("a")


In [34]:
len(connections_links)

102

In [35]:
#initializing logs file
logging.basicConfig(filename='linkedin.log',level=logging.DEBUG,
                    format='%(asctime)s:%(levelname)s:%(message)s')

In [41]:
info = crawler(driver= d, links =connections_links)

In [44]:
# info[-2]

['Amir Hassan Mousavi',
 'Digikala Group Corporate Communications Senior Manager',
 'Tehran, Iran',
 'https://www.linkedin.com/in/amirhassanmousavi/']

In [46]:
dump_data(info)

dummped all contacts info Successfully
